## Autonomous Database(23ai) をベクトルストアとして活用するためのセットアップ

このノートブックでは、Autonomous Database をベクトルストアとして活用するためのセットアップを行います。  

### 任意: 事前準備

Python で実装したコードから ADB に接続するために必要な Wallet のダウンロードと展開を行います。  

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

autonomous_database_id = os.getenv("AUTONOMOUS_DATABASE_ID")
wallet_dir = os.getenv("WALLET_DIR")
wallet_password = os.getenv("WALLET_PASSWORD")

In [ ]:
import shutil
from oci.auth.signers import InstancePrincipalsSecurityTokenSigner
from oci.database.database_client import DatabaseClient
from oci.database.models import GenerateAutonomousDatabaseWalletDetails

signer = InstancePrincipalsSecurityTokenSigner()
database_client = DatabaseClient(config={}, signer=signer)
res = database_client.generate_autonomous_database_wallet(
    autonomous_database_id=autonomous_database_id,
    generate_autonomous_database_wallet_details=GenerateAutonomousDatabaseWalletDetails(
        generate_type="ALL",
        password=wallet_password
    )
)

data = res.data

zip_file = os.path.join(wallet_dir, 'wallet.zip')
os.makedirs(wallet_dir, exist_ok=True)
with open(zip_file, 'wb') as f:
    for chunk in data.raw.stream(1024 * 1024, decode_content=False):
        f.write(chunk)
shutil.unpack_archive(zip_file, wallet_dir)
print("zip file was unpacked in", wallet_dir)

### Oracle Database とのコネクション作成

コネクションの作成に必要なパラメータを環境変数から読み込みます。

In [ ]:
import oracledb

_ = load_dotenv(find_dotenv())
username = os.getenv("USERNAME")
password = os.getenv("PASSWORD")
dsn = os.getenv("DSN")
config_dir = os.getenv("CONFIG_DIR")
wallet_dir = os.getenv("WALLET_DIR")
wallet_password = os.getenv("WALLET_PASSWORD")
table_name = os.getenv("TABLE_NAME")

Oracle Database とのコネクションを作成します。  
Jupyter Notebook のため、`with` 句は利用していませんが、アプリケーションに組み込む際は、コネクションが確実にクローズされることを保証するために、`with` 句を用いてコネクションを作成ください。

参考: [https://python-oracledb.readthedocs.io/en/latest/user_guide/connection_handling.html#closing-connections](https://python-oracledb.readthedocs.io/en/latest/user_guide/connection_handling.html#closing-connections)

In [ ]:
connection = oracledb.connect(
    dsn=dsn,
    user=username,
    password=password,
    config_dir=config_dir,
    wallet_location=wallet_dir,
    wallet_password=wallet_password
)

### ドキュメントの読み込み + チャンク分割

対象のドキュメント（日本オラクルの有価証券報告書）を読み込み、埋め込み表現を得るためにチャンクに分割します。

In [ ]:
import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders.oracleai import OracleTextSplitter, OracleDocReader, OracleDocLoader

params = {"split": "recursively", "max": 300, "by": "words", "overlap": 30, "normalize": "all"}
character_splitter = OracleTextSplitter(
    conn=connection,
    params=params
)
files = glob.glob("../data/*.pdf")
docs = []
for file in files:
    loader = PyPDFLoader(file_path=file)
    pages = loader.load_and_split(text_splitter=character_splitter)
    docs.extend(pages)
    for page in pages:
        print("content length:", len(page.page_content))
print("docs size:", len(docs))

### ADB　へのデータ投入

データベースに格納する際のベクトルを得るための埋め込み関数とベクトルデータベースを作成します。

In [ ]:
from langchain_community.embeddings.oci_generative_ai import OCIGenAIEmbeddings
from langchain_community.vectorstores.oraclevs import OracleVS
from langchain_community.vectorstores.utils import DistanceStrategy

_ = load_dotenv(find_dotenv())
compartment_id = os.getenv("COMPARTMENT_ID")
service_endpoint = os.getenv("SERVICE_ENDPOINT")

embedding_function = OCIGenAIEmbeddings(
    auth_type="INSTANCE_PRINCIPAL",
    model_id="cohere.embed-multilingual-v3.0",
    service_endpoint=service_endpoint,
    compartment_id=compartment_id,
)

oracle_vs = OracleVS(
    client=connection,
    embedding_function=embedding_function,
    table_name=table_name,
    distance_strategy=DistanceStrategy.COSINE,
    query="What is Oracle Database?"
)


チャンク分割したドキュメントデータを Oracle Database に格納します。

In [ ]:
oracle_vs.add_documents(documents=docs)

## Vector Search

自然言語での類似検索実行例

In [ ]:
query = "日本オラクルの財政状況を教えてください。"

res = oracle_vs.similarity_search(query=query)
print(res)

自然言語での類似検索に加え関連度のスコア情報も取得する例  
**注: langchain-community 0.2.6 時点では本関数（正確には、`_select_relevance_score_fn`）は未実装のため実行不可**

In [ ]:
res = oracle_vs.similarity_search_with_relevance_scores(query=query)
print(res)

## LLM(chat feature) を組み合わせた対話形式の問い合わせ

In [ ]:
from langchain_community.chat_models.oci_generative_ai import ChatOCIGenAI

chat = ChatOCIGenAI(
    auth_type="INSTANCE_PRINCIPAL",
    service_endpoint=service_endpoint,
    compartment_id=compartment_id,
    model_id="cohere.command-r-plus",
    is_stream=True,
    model_kwargs={
        "temperature": 0,
        "max_tokens": 2500,
        "top_p": 0.75,
        "top_k": 0,
        "frequency_penalty": 0,
        "presence_penalty": 0
    }
)

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """
可能な限り、得られたコンテキストに基づいて以下の質問に対する回答を作成してください。

## コンテキスト
{context}

## 質問
{question}
"""

prompt_template = PromptTemplate.from_template(
    template=template,
)

chain = (
    {"context": oracle_vs.as_retriever(), "question": RunnablePassthrough()}
    | prompt_template
    | chat
    | StrOutputParser()
)

In [ ]:
query = "日本オラクルの有価証券報告書を読むのが大変なので簡潔にまとめてください。"

res = chain.stream(query)

for chunk in res:
    print(chunk, end="")